In [1]:
import os
import tensorflow
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np

print('complete')

Using TensorFlow backend.


complete


In [5]:
clinical_data = pd.read_csv(r'D:\Transcend_(E)\_IramS\Covid Challenge\only_images.csv',delimiter=',',header=0)
clinical_data.sort_values(by='ImageFile')

,PatientID,ImageFile,Hospital,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,Fibrinogen,LDH,Ddimer,Ox_percentage,PaO2,SaO2,pH,CardiovascularDisease,RespiratoryFailure,Prognosis
23,P_1,P_1.png,A,38.0,NaN,NaN,NaN,0.0,NaN,0.28,NaN,NaN,NaN,96.0,NaN,NaN,7.41,0.0,NaN,0
49,P_10,P_10.png,A,76.0,1.0,36.7,NaN,0.0,8.88,NaN,NaN,1159.0,NaN,NaN,NaN,90.0,7.49,NaN,0.0,0
50,P_101,P_101.png,A,78.0,NaN,NaN,1.0,1.0,4.66,NaN,NaN,NaN,NaN,90.0,NaN,92.0,7.50,1.0,NaN,0
0,P_102,P_102.png,A,NaN,1.0,37.3,1.0,0.0,5.37,3.96,NaN,NaN,NaN,93.0,63.0,93.0,7.46,0.0,0.0,0
72,P_103,P_103.png,A,68.0,0.0,NaN,1.0,1.0,NaN,1.81,NaN,NaN,NaN,97.0,NaN,NaN,NaN,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,P_93,P_93.png,A,62.0,0.0,36.6,NaN,0.0,NaN,0.80,NaN,NaN,NaN,97.0,70.0,NaN,NaN,0.0,NaN,0
32,P_94,P_94.png,A,35.0,1.0,NaN,1.0,1.0,3.99,NaN,NaN,NaN,NaN,NaN,77.0,NaN,7.44,0.0,0.0,0
39,P_96,P_96.png,A,NaN,0.0,39.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,97.0,69.0,NaN,7.55,0.0,0.0,0
65,P_97,P_97.png,A,67.0,0.0,38.3,1.0,1.0,3.98,0.97,NaN,NaN,NaN,96.0,NaN,96.0,NaN,NaN,0.0,0


In [6]:
from pathlib import Path 
test_images_directory = str(Path(r'D:\Transcend_(E)\_IramS\Covid Challenge\threshold_otsu_test'))

test_images_path = [(test_images_directory + "\\"+  f) for f in clinical_data['ImageFile'] if isfile(join(test_images_directory,f))]

In [7]:
base_model = DenseNet121(include_top=False, pooling='avg', weights='imagenet', input_shape= (224, 224, 3))
x = base_model.output
x_flatten = Flatten()(x)
model = Model(inputs=base_model.input, outputs=x_flatten)

In [ ]:
def process_image(path):
    img = image.load_img(path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

densnet_feature_list = []

for path in (test_images_path):
    img_data = process_image(path)
    densnet_feature = model.predict(img_data)
    densnet_feature_np = densnet_feature
    densnet_feature_list.append(densnet_feature_np)

In [ ]:
list_densnet_feature_list= [l.tolist() for l in densnet_feature_list]

In [ ]:
feature_df = pd.DataFrame(list_densnet_feature_list, columns=['Features'])

In [ ]:
feature_df.index = clinical_data.ImageFile

In [ ]:
feature_df.to_csv('D:\Transcend_(E)\_IramS\Covid Challenge\Densnet_Threshold_Image_features_test.csv', header=True)